In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline

import re

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

#to see which classes have been defined so far
[modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

loc = "angular_distr_el/CH9/ID_ALL_mol_e0_valid/EN_gate/MFPADs_multinew_std"

fileRCR = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\R-C3H3F3O_550eV_CR_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
fileRCL = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\R-C3H3F3O_550eV_CL_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
k=fileRCR[loc].keys()
d=dict(fileRCR[loc].classes()) # shows the type of each item
dict(fileRCR[loc]) # keys : values
check = fileRCR[loc+"/MFPAD3D_engate_costheta_-1.00_phi_-180"]
type(check)
check._members()

In [ ]:
b=check.numpy()
type(b)
b

# Structure of values inside TH2D. MFPAD_xxx is a list

r_norm_nhistRCR = nhistRCR[0] / nhistRCR[0].sum(axis=1)[:,np.newaxis] # r values normalise to 1

phi_nhistRCR = nhistRCR[1][0][0] # phi

costheta_nhistRCR = nhistRCR[1][0][1] # cos(theta)

In [ ]:
# initialize a nparray: best way is to know the size in advance
# MFPAD_RCR=np.zeros((72,2))
# ctetha_RCR=np.zeros((72,2))

MFPAD_RCR=[] # initilialises a list
ctetha_RCR=[]
cosphi=[]
MFPAD_xy=[]
run_one=0
for key, value in fileRCR[loc].items():
        filename=loc+"/"+str(key).split(";")[0].replace("b'","")
        if "mfpad3d_engate_costheta" in filename.lower():
            # print(filename)
            ctheta_nm=float((str(key).split("costheta_")[1]).split("_phi")[0])
            phi_nm=float((str(key).split("phi_")[1]).split(";")[0])
            # phicos.append((ctheta_nm,phi_nm))
            cosphi.append((ctheta_nm+0.1666,phi_nm+15)) # centered in the middle of the bins
            #ctheta_n=re.search("costheta_(.*)_phi", str(key)).group(1) #alternative with import re
            temp=np.array(fileRCR[filename].numpy())
            MFPAD_RCR.append(temp[0]) # it is a list!
            if run_one == 0:
                MFPAD_xy.append((temp[1][0][0] , temp[1][0][1])) # phi cos(theta) from 2D
                run_one=1
        elif "cos(theta)" in filename.lower():
            ctheta_nc=float((str(key).split("costheta_")[1]).split("_phi")[0])
            temp=np.array(fileRCR[filename].numpy())
            ctetha_RCR.append(temp[0]) # it is a list!
            ctheta_c = temp[1] # cos(theta) from 1D
MFPAD_RCR=np.array(MFPAD_RCR)
ctetha_RCR=np.array(ctetha_RCR) # it is a list!


In [ ]:
MFPAD_RCL=[] # initilialises a list
ctetha_RCL=[]
cosphi=[]
MFPAD_xy=[]
run_one=0
for key, value in fileRCL[loc].items():
        filename=loc+"/"+str(key).split(";")[0].replace("b'","")
        if "mfpad3d_engate_costheta" in filename.lower():
            # print(filename)
            ctheta_nm=float((str(key).split("costheta_")[1]).split("_phi")[0])
            phi_nm=float((str(key).split("phi_")[1]).split(";")[0])
            # phicos.append((ctheta_nm,phi_nm)) #they are inverted
            cosphi.append((ctheta_nm+0.1666,phi_nm+15))  # centered in the middle of the bins
            #ctheta_n=re.search("costheta_(.*)_phi", str(key)).group(1) #alternative with import re
            temp=np.array(fileRCL[filename].numpy())
            MFPAD_RCL.append(temp[0]) # it is a list!
            if run_one == 0:
                MFPAD_xy.append((temp[1][0][0] , temp[1][0][1])) # phi cos(theta) from 2D
                run_one=1
        elif "cos(theta)" in filename.lower():
            ctheta_nc=float((str(key).split("costheta_")[1]).split("_phi")[0])
            temp=np.array(fileRCL[filename].numpy())
            ctetha_RCL.append(temp[0]) # it is a list!
            ctheta_c = temp[1] # cos(theta) from 1D
MFPAD_RCL=np.array(MFPAD_RCL)
ctetha_RCL=np.array(ctetha_RCL) # it is a list!

In [ ]:
test=fileRCL[loc+"/cos(theta)_e[0]_costheta_-1.00_phi_-180"]
# test.__dir__()
# print(test._fTsumwx2)
# print(test.variances)
print(test.values)
errc = np.sqrt(test.values)
print(errc)

In [ ]:
plt.scatter(range(len(test.values)),test.values,color='r')
plt.errorbar(range(len(test.values)),test.values, yerr=errc, fmt="o")

In [ ]:
print(MFPAD_xy[0][0].shape)
MFPAD_xy[0][0] #to be reduced like ctheta_c_red

In [ ]:
print(ctetha_RCR[0].shape)
ctetha_RCR[0]

In [ ]:
#need to take the centre
print(ctheta_c.shape)
ctheta_c

In [ ]:
ctheta_cred = (ctheta_c[1:] + ctheta_c[:-1])/2
print(ctheta_cred.shape)

In [ ]:
print(type(temp[0]),temp[0].shape, len(MFPAD_RCR))
temp[0]
temp[1]

In [ ]:
cPECD_norm =  np.divide(np.subtract(ctetha_RCR/ctetha_RCR.sum(axis=1)[:,np.newaxis],ctetha_RCL/ctetha_RCL.sum(axis=1)[:,np.newaxis]),np.add(ctetha_RCR/ctetha_RCR.sum(axis=1)[:,np.newaxis],ctetha_RCL/ctetha_RCL.sum(axis=1)[:,np.newaxis]))

cPECD =  np.divide(np.subtract(ctetha_RCR,ctetha_RCL),np.add(ctetha_RCR,ctetha_RCL))

# from the statistical book var = sqrt(1/Ntot)

errPECD = np.sqrt(1/(ctetha_RCR+ctetha_RCL))

cPECD.shape

In [ ]:
plt.scatter(ctheta_cred,cPECD_norm[0])

In [ ]:
# everything in iminuit is done through the Minuit object, so we import it
from iminuit import Minuit
# we also need a cost function to fit and import the LeastSquares function
from iminuit.cost import LeastSquares
from iminuit.util import describe, make_func_code
import traceback

class LeastSquares:
    """
    Generic least-squares cost function.
    """
    def __init__(self, model, x, y):
        self.model = model  # model predicts y for given x
        self.x = np.array(x)
        self.y = np.array(y)

    def __call__(self, *par):  # we accept a variable number of model parameters
        ym = self.model(self.x, *par)
        chi2 = np.sum((self.y - ym)**2)
        return chi2

# does this inherits from LeastSquare?
class BetterLeastSquares(LeastSquares):
    def __init__(self, model, x, y):
        super().__init__(model, x, y)
        self.func_code = make_func_code(describe(model)[1:])

#NOTE +1 IS NOT NECESSARY!!!
def PECD6(x, b1, b2, b3, b4, b5, b6):
    return ((b1*x + 0.5*b3*(5*x**3-3*x) +0.125*b5*(63*x**5-70*x**3+15*x)) / (1 + 0.5*b2*(3*x**2-1) + 0.125*b4*(35*x**4-30**2+3) + 0.0625*b6*(231*x**6-315*x**4+105*x**2-5)))

x_data = ctheta_cred
y_data = cPECD_norm[0]

# class iminuit.cost.LeastSquares(x, y, yerror, model, loss='linear', verbose=0)
# Use this if you have data of the form (x, y +/- yerror).
lsq = BetterLeastSquares(PECD6, x_data, y_data,)

m = Minuit(lsq, errordef=Minuit.LEAST_SQUARES, limit_b1=(-1, 1), limit_b2=(-1, 1), limit_b3=(-1, 1), limit_b4=(-1, 1), limit_b5=(-1, 1), limit_b6=(-1, 1)) # initial values are 0 by default
# m = Minuit(lsq, error_b1=0.01, error_b2=0.01, limit=((-1, 1), (-1, 1)), name=("b1", "b2"), pedantic=False)
m.migrad()
# print(m.params)

In [ ]:
# modified with error
# everything in iminuit is done through the Minuit object, so we import it
from iminuit import Minuit
# we also need a cost function to fit and import the LeastSquares function
from iminuit.cost import LeastSquares
from iminuit.util import describe, make_func_code
import traceback

class LeastSquares:
    """
    Generic least-squares cost function with error.
    """
    def __init__(self, model, x, y, err):
        self.model = model  # model predicts y for given x
        self.x = np.array(x)
        self.y = np.array(y)
        self.err = np.array(err)
        # self.err = err

    def __call__(self, *par):  # we accept a variable number of model parameters
        ym = self.model(self.x, *par)
        chi2 = np.sum((self.y - ym)**2/(self.err**2))
        return chi2

# does this inherits from LeastSquare?
class BetterLeastSquares(LeastSquares):
    def __init__(self, model, x, y, err):
        super().__init__(model, x, y, err)
        self.func_code = make_func_code(describe(model)[1:])

#NOTE +1 IS NOT NECESSARY!!!
def PECD6(x, b1, b2, b3, b4, b5, b6):
    return ((b1*x + 0.5*b3*(5*x**3-3*x) +0.125*b5*(63*x**5-70*x**3+15*x)) / (1 + 0.5*b2*(3*x**2-1) + 0.125*b4*(35*x**4-30**2+3) + 0.0625*b6*(231*x**6-315*x**4+105*x**2-5)))

x_data = ctheta_cred
y_data = cPECD_norm[0]
y_err = errPECD[0]

# class iminuit.cost.LeastSquares(x, y, yerror, model, loss='linear', verbose=0)
# Use this if you have data of the form (x, y +/- yerror).
lsq = BetterLeastSquares(PECD6, x_data, y_data, y_err)

# m = Minuit(lsq, errordef=Minuit.LEAST_SQUARES,limit_b1=(-1, 1), limit_b2=(-1, 1), limit_b3=(-1, 1), limit_b4=(-1, 1), limit_b5=(-1, 1), limit_b6=(-1, 1)) # initial values are 0 by default
m = Minuit(lsq, errordef=Minuit.LEAST_SQUARES, limit_b1=(-1, 1), limit_b2=(-1, 1))
m.migrad()
m.hesse() # run covariance estimator
# print(m.params)

In [ ]:
from scipy.interpolate import make_interp_spline, BSpline
x_data_new = np.linspace(x_data.min(), x_data.max(), 300)
bspl = make_interp_spline(x_data, PECD6(x_data, *m.values.values()), k=5)
plt.plot(x_data_new, bspl(x_data_new))
plt.scatter(ctheta_cred,cPECD_norm[0],color='r')
plt.errorbar(ctheta_cred,cPECD_norm[0], yerr=errPECD[0], fmt="o")
plt.show()
for p in m.parameters:
    print("{} = {:2.3e} +/- {:2.3e}".format(p, m.values[p], m.errors[p]))

In [ ]:
# modified with error
# everything in iminuit is done through the Minuit object, so we import it
from iminuit import Minuit
# we also need a cost function to fit and import the LeastSquares function
from iminuit.cost import LeastSquares
from iminuit.util import describe, make_func_code
import traceback

class LeastSquares:
    """
    Generic least-squares cost function with error.
    """
    def __init__(self, model, x, y, err):
        self.model = model  # model predicts y for given x
        self.x = np.array(x)
        self.y = np.array(y)
        self.err = np.array(err)
        # self.err = err

    def __call__(self, *par):  # we accept a variable number of model parameters
        ym = self.model(self.x, *par)
        chi2 = np.sum((self.y - ym)**2/(self.err**2))
        return chi2

# does this inherits from LeastSquare?
class BetterLeastSquares(LeastSquares):
    def __init__(self, model, x, y, err):
        super().__init__(model, x, y, err)
        self.func_code = make_func_code(describe(model)[1:])

# param_matrix=np.zeros((6,72)) #nparray initialization
param_matrix = [[0 for i in range(72)] for j in range(6)] #i col, j row

fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex=True , sharey=True)
fig.subplots_adjust(hspace = .5, wspace=.5)
custom_xlim = (-1, 1)
custom_ylim = (-0.2,0.2)

for i in range(len(cosphi)): # len = 72
    #print(i)
    x_data = ctheta_cred
    y_data = cPECD_norm[i]
    y_err = errPECD[i]

    lsq = BetterLeastSquares(PECD6, x_data, y_data, y_err)

    m = Minuit(lsq, errordef=Minuit.LEAST_SQUARES, limit_b1=(-1, 1), limit_b2=(-1, 1), limit_b3=(-1, 1), limit_b4=(-1, 1), limit_b5=(-1, 1), limit_b6=(-1, 1)) # initial values are 0 by default
    # m = Minuit(lsq, error_b1=0.01, error_b2=0.01, limit=((-1, 1), (-1, 1)), name=("b1", "b2"), pedantic=False)
    m.migrad() # run optimiser
    m.hesse() # run covariance estimator
    # print(m.params)
    x_data_new = np.linspace(x_data.min(), x_data.max(), 300)
    bspl = make_interp_spline(x_data, PECD6(x_data, *m.values.values()), k=5)
    
    ax = fig.add_subplot(6, 12, i+1)
    ax.plot(x_data_new, bspl(x_data_new))
    ax.scatter(ctheta_cred,y_data,color='r')
    ax.set_xlim(custom_xlim)
    ax.set_ylim(custom_ylim)
    ax.errorbar(ctheta_cred,y_data, yerr=y_err, fmt="o")
    # ax.axis('off')
    for j,p in zip(range(len(m.parameters)),m.parameters): # len = 6
        param_matrix[j][i] = ((m.values[p], m.errors[p]))
        # param_matrix[i][j] = (m.values[p], m.errors[p])

fig.tight_layout()
plt.show()
    # for p in m.parameters:
    #     print("{} = {:2.3e} +/- {:2.3e}".format(p, m.values[p], m.errors[p]))

In [ ]:
import pprint
pprint.pprint(param_matrix)

In [ ]:
len(m.parameters)

In [ ]:
test2=[col[0] for col in param_matrix] #list
b1=[col[0] for col in test2]

In [ ]:
test=np.array(param_matrix) #numpy
b1 = test[0,:,0]
b2 = test[1,:,0]

In [ ]:
b1[1]

In [ ]:
fig,axes = plt.subplots(1,6, figsize=(50, 7.5))
cosphi=np.array(cosphi)
for i in range(6):
    # ax[0,1] = fig.add_subplot(1, 2, 1)
    axes[i].contourf(cosphi[:,1].reshape(12,6),cosphi[:,0].reshape(12,6),test[i,:,0].reshape(12,6), np.arange(-1., 1., .005), extend='both', cmap='seismic')
    # axes[i].contourf(cosphi[:,0].reshape(12,6),cosphi[:,1].reshape(12,6),test[i,:,0].reshape(6,12).T)
plt.show()

In [ ]:
import matplotlib as mpl
from matplotlib import cm

magma_cmap = mpl.cm.get_cmap('magma')
seismic_cmap = mpl.cm.get_cmap('seismic')

magma_rgb = []
seismic_rgb = []
norm = mpl.colors.Normalize(vmin=0, vmax=255)

for i in range(0, 255):
       k = mpl.colors.colorConverter.to_rgb(magma_cmap(norm(i)))
       magma_rgb.append(k)

for i in range(0, 255):
       k = mpl.colors.colorConverter.to_rgb(seismic_cmap(norm(i)))
       seismic_rgb.append(k)

def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

Magma_r = matplotlib_to_plotly(magma_cmap, 255)
Seismic_r = matplotlib_to_plotly(seismic_cmap, 255)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2,
                    subplot_titles=('Without Smoothing', 'With Smoothing'))

fig.add_trace(go.Contour(z=test[0,:,0], x=cosphi[:,1], y=cosphi[:,0], line_smoothing=0, colorscale=Seismic_r, contours=dict(start=-0.2, end=0.2, size=0.04)), 1, 1)
fig.add_trace(go.Contour(z=test[0,:,0], x=cosphi[:,1], y=cosphi[:,0], line_smoothing=0.75, colorscale=Seismic_r,contours=dict(start=-0.2, end=0.2, size=0.04)), 1, 2)


fig.show()

In [ ]:
len(test)

In [ ]:
#NOTE: THE SCALE IS FAKE FOR PLOT 2 AND 6

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = go.Figure()

fig = make_subplots(rows=6, cols=2, vertical_spacing=0.015,
                    subplot_titles=('Without Smoothing', 'With Smoothing'))

for i in range(len(test)):
    if i==1 or i==5:
        fig.add_trace(go.Contour(z=test[i,:,0], x=cosphi[:,1], y=cosphi[:,0], line_smoothing=0, colorscale=Seismic_r, legendgroup="no", showscale=False, showlegend=False, contours=dict(start=-1, end=1, size=0.5)), i+1, 1)
        fig.add_trace(go.Contour(z=test[i,:,0], x=cosphi[:,1], y=cosphi[:,0], line_smoothing=0.85, colorscale=Seismic_r, legendgroup="no", showscale=False, showlegend=False, contours=dict(start=-1, end=1, size=0.5)), i+1, 2)
    else:
        fig.add_trace(go.Contour(z=test[i,:,0], x=cosphi[:,1], y=cosphi[:,0], line_smoothing=0, colorscale=Seismic_r, legendgroup="yes", contours=dict(start=-0.2, end=0.2, size=0.04)), i+1, 1)
        fig.add_trace(go.Contour(z=test[i,:,0], x=cosphi[:,1], y=cosphi[:,0], line_smoothing=0.85, colorscale=Seismic_r, legendgroup="yes", contours=dict(start=-0.2, end=0.2, size=0.04)), i+1, 2)

fig.update_layout(
    # coloraxis=dict(colorscale=Seismic_r),
    # showlegend=False,
    autosize=False,
    # width=800,
    height=1500,
    margin=dict(l=0,r=0,b=25,t=25)
    )

fig.show()